In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7/python"
os.environ["JAVA_HOME"] = "/usr/java/jdk1.8.0_161/jre"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Cricket").getOrCreate()

In [4]:
ball = spark.read.csv("Ball_By_Ball.csv",header = True, inferSchema = True)

## Task 

#### Find the number of 6s hit in Wankhede stadium

Let's expore the data and find what columns we need to answer this question 

In [7]:
ball.columns

['MatcH_id',
 'Over_id',
 'Ball_id',
 'Innings_No',
 'Team_Batting',
 'Team_Bowling',
 'Striker_Batting_Position',
 'Extra_Type',
 'Runs_Scored',
 'Extra_runs',
 'Wides',
 'Legbyes',
 'Byes',
 'Noballs',
 'Penalty',
 'Bowler_Extras',
 'Out_type',
 'Caught',
 'Bowled',
 'Run_out',
 'LBW',
 'Retired_hurt',
 'Stumped',
 'caught_and_bowled',
 'hit_wicket',
 'ObstructingFeild',
 'Bowler_Wicket',
 'Match_Date',
 'Season',
 'Striker',
 'Non_Striker',
 'Bowler',
 'Player_Out',
 'Fielders',
 'Striker_match_SK',
 'StrikerSK',
 'NonStriker_match_SK',
 'NONStriker_SK',
 'Fielder_match_SK',
 'Fielder_SK',
 'Bowler_match_SK',
 'BOWLER_SK',
 'PlayerOut_match_SK',
 'BattingTeam_SK',
 'BowlingTeam_SK',
 'Keeper_Catch',
 'Player_out_sk',
 'MatchDateSK']

#### The data has lot of columns. Instead of carring all the columns lets select the relevant columns to solve the problem

This data set has runs scored and number of extras these are relevant columns. But we still have no way to get location of ground. let's expolre other data sets. 

In [8]:
match = spark.read.csv("Match.csv", header = True, inferSchema = True)

In [9]:
match.columns

['Match_SK',
 'match_id',
 'Team1',
 'Team2',
 'match_date',
 'Season_Year',
 'Venue_Name',
 'City_Name',
 'Country_Name',
 'Toss_Winner',
 'match_winner',
 'Toss_Name',
 'Win_Type',
 'Outcome_Type',
 'ManOfMach',
 'Win_Margin',
 'Country_id']

### This column dadaset has a column called Venue_Name

we can join both this tables to find our answer. 

In [10]:
ball_data = ball.select([
 'MatcH_id',
 'Over_id',
 'Ball_id',
 'Extra_Type',
 'Runs_Scored']) 

In [12]:
ball_data.show()

+--------+-------+-------+----------+-----------+
|MatcH_id|Over_id|Ball_id|Extra_Type|Runs_Scored|
+--------+-------+-------+----------+-----------+
|  598028|     15|      6| No Extras|          4|
|  598028|     14|      1| No Extras|          1|
|  598028|     14|      2| No Extras|          1|
|  598028|     14|      3| No Extras|          1|
|  598028|     14|      4| No Extras|          0|
|  598028|     14|      5| No Extras|          4|
|  598028|     14|      6| No Extras|          2|
|  598028|     13|      1| No Extras|          1|
|  598028|     13|      2| No Extras|          4|
|  598028|     13|      3| No Extras|          1|
|  598028|     13|      4| No Extras|          0|
|  598028|     13|      5| No Extras|          0|
|  598028|     13|      6| No Extras|          1|
|  598028|     12|      1| No Extras|          1|
|  598028|     12|      2| No Extras|          0|
|  598028|     12|      3| No Extras|          1|
|  598028|     12|      4| No Extras|          1|


In [13]:
match_data = match.select(['match_id','Venue_Name'])

In [14]:
match_data.show()

+--------+--------------------+
|match_id|          Venue_Name|
+--------+--------------------+
|  335987|M Chinnaswamy Sta...|
|  335988|Punjab Cricket As...|
|  335989|    Feroz Shah Kotla|
|  335990|    Wankhede Stadium|
|  335991|        Eden Gardens|
|  335992|Sawai Mansingh St...|
|  335993|Rajiv Gandhi Inte...|
|  335994|MA Chidambaram St...|
|  335995|Rajiv Gandhi Inte...|
|  335996|Punjab Cricket As...|
|  335997|M Chinnaswamy Sta...|
|  335998|MA Chidambaram St...|
|  335999|Dr DY Patil Sport...|
|  336000|Punjab Cricket As...|
|  336001|M Chinnaswamy Sta...|
|  336002|        Eden Gardens|
|  336003|    Feroz Shah Kotla|
|  336004|Rajiv Gandhi Inte...|
|  336005|Sawai Mansingh St...|
|  336006|MA Chidambaram St...|
+--------+--------------------+
only showing top 20 rows



In [15]:
ball_data.createOrReplaceTempView("ball")

In [16]:
match_data.createOrReplaceTempView("match")

In [17]:
Query1 = "ball.MatcH_id, ball.Extra_Type, ball.Runs_Scored, match.Venue_Name"

In [20]:
ball_data.count()

150451

In [21]:
match_data.count()

637

In [18]:
df = spark.sql("Select "+Query1+" FROM ball LEFT JOIN match ON ball.MatcH_id = match.MatcH_id ")

In [19]:
df.show()

+--------+----------+-----------+--------------------+
|MatcH_id|Extra_Type|Runs_Scored|          Venue_Name|
+--------+----------+-----------+--------------------+
|  598028| No Extras|          4|M Chinnaswamy Sta...|
|  598028| No Extras|          1|M Chinnaswamy Sta...|
|  598028| No Extras|          1|M Chinnaswamy Sta...|
|  598028| No Extras|          1|M Chinnaswamy Sta...|
|  598028| No Extras|          0|M Chinnaswamy Sta...|
|  598028| No Extras|          4|M Chinnaswamy Sta...|
|  598028| No Extras|          2|M Chinnaswamy Sta...|
|  598028| No Extras|          1|M Chinnaswamy Sta...|
|  598028| No Extras|          4|M Chinnaswamy Sta...|
|  598028| No Extras|          1|M Chinnaswamy Sta...|
|  598028| No Extras|          0|M Chinnaswamy Sta...|
|  598028| No Extras|          0|M Chinnaswamy Sta...|
|  598028| No Extras|          1|M Chinnaswamy Sta...|
|  598028| No Extras|          1|M Chinnaswamy Sta...|
|  598028| No Extras|          0|M Chinnaswamy Sta...|
|  598028|

In [23]:
df.collect()

[Row(MatcH_id=598028, Extra_Type='No Extras', Runs_Scored=4, Venue_Name='M Chinnaswamy Stadium'),
 Row(MatcH_id=598028, Extra_Type='No Extras', Runs_Scored=1, Venue_Name='M Chinnaswamy Stadium'),
 Row(MatcH_id=598028, Extra_Type='No Extras', Runs_Scored=1, Venue_Name='M Chinnaswamy Stadium'),
 Row(MatcH_id=598028, Extra_Type='No Extras', Runs_Scored=1, Venue_Name='M Chinnaswamy Stadium'),
 Row(MatcH_id=598028, Extra_Type='No Extras', Runs_Scored=0, Venue_Name='M Chinnaswamy Stadium'),
 Row(MatcH_id=598028, Extra_Type='No Extras', Runs_Scored=4, Venue_Name='M Chinnaswamy Stadium'),
 Row(MatcH_id=598028, Extra_Type='No Extras', Runs_Scored=2, Venue_Name='M Chinnaswamy Stadium'),
 Row(MatcH_id=598028, Extra_Type='No Extras', Runs_Scored=1, Venue_Name='M Chinnaswamy Stadium'),
 Row(MatcH_id=598028, Extra_Type='No Extras', Runs_Scored=4, Venue_Name='M Chinnaswamy Stadium'),
 Row(MatcH_id=598028, Extra_Type='No Extras', Runs_Scored=1, Venue_Name='M Chinnaswamy Stadium'),
 Row(MatcH_id=598028

In [22]:
df.count()

150451

In [24]:
df.createOrReplaceTempView("6data")

Total no of 6 hit in Wankhede Stadium Mumbai 

In [25]:
Query2 = "Venue_Name=='Wankhede Stadium' AND Extra_Type=='No Extras' AND Runs_Scored==6"

In [26]:
spark.sql("SELECT * FROM 6data WHERE "+Query2).count()

649

Find Highest score made to which venue?

In [27]:
most_score = spark.sql("SELECT Venue_Name, sum(Runs_Scored) as total_score FROM 6data group by Venue_Name")

In [28]:
most_score.show()

+--------------------+-----------+
|          Venue_Name|total_score|
+--------------------+-----------+
|Dubai Internation...|       1942|
|Himachal Pradesh ...|       2716|
|Sardar Patel Stad...|       3589|
|Punjab Cricket As...|      10421|
|    Barabati Stadium|       2195|
|Punjab Cricket As...|       1184|
|Punjab Cricket As...|       2232|
|Rajiv Gandhi Inte...|       2419|
|       Nehru Stadium|       1286|
|Maharashtra Crick...|       4538|
|        Eden Gardens|      17105|
|     OUTsurance Oval|        484|
|M Chinnaswamy Sta...|      19423|
|    Feroz Shah Kotla|      17491|
|Rajiv Gandhi Inte...|      11748|
|   Brabourne Stadium|       3595|
|Vidarbha Cricket ...|        822|
|          Green Park|       1236|
|Holkar Cricket St...|       1487|
|Shaheed Veer Nara...|       1664|
+--------------------+-----------+
only showing top 20 rows



In [29]:
most_score.collect()

[Row(Venue_Name='Dubai International Cricket Stadium', total_score=1942),
 Row(Venue_Name='Himachal Pradesh Cricket Association Stadium', total_score=2716),
 Row(Venue_Name='Sardar Patel Stadium, Motera', total_score=3589),
 Row(Venue_Name='Punjab Cricket Association Stadium, Mohali', total_score=10421),
 Row(Venue_Name='Barabati Stadium', total_score=2195),
 Row(Venue_Name='Punjab Cricket Association IS Bindra Stadium Mohali', total_score=1184),
 Row(Venue_Name='Punjab Cricket Association IS Bindra Stadium, Mohali', total_score=2232),
 Row(Venue_Name='Rajiv Gandhi International Stadium Uppal', total_score=2419),
 Row(Venue_Name='Nehru Stadium', total_score=1286),
 Row(Venue_Name='Maharashtra Cricket Association Stadium', total_score=4538),
 Row(Venue_Name='Eden Gardens', total_score=17105),
 Row(Venue_Name='OUTsurance Oval', total_score=484),
 Row(Venue_Name='M Chinnaswamy Stadium', total_score=19423),
 Row(Venue_Name='Feroz Shah Kotla', total_score=17491),
 Row(Venue_Name='Rajiv Gand

In [30]:
most_score.createOrReplaceTempView("most_score_table")

In [31]:
spark.sql("select * from most_score_table order by total_score desc").show()

+--------------------+-----------+
|          Venue_Name|total_score|
+--------------------+-----------+
|M Chinnaswamy Sta...|      19423|
|    Feroz Shah Kotla|      17491|
|        Eden Gardens|      17105|
|    Wankhede Stadium|      16996|
|MA Chidambaram St...|      14471|
|Rajiv Gandhi Inte...|      11748|
|Punjab Cricket As...|      10421|
|Sawai Mansingh St...|       9231|
|Maharashtra Crick...|       4538|
|Subrata Roy Sahar...|       4518|
|Dr DY Patil Sport...|       4490|
|           Kingsmead|       4076|
|   Brabourne Stadium|       3595|
|Sardar Patel Stad...|       3589|
|     SuperSport Park|       3440|
|Saurashtra Cricke...|       3198|
|Dr. Y.S. Rajasekh...|       2995|
|Himachal Pradesh ...|       2716|
|Rajiv Gandhi Inte...|       2419|
|Punjab Cricket As...|       2232|
+--------------------+-----------+
only showing top 20 rows

